In [19]:
import pandas as pd
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.firefox import GeckoDriverManager

In [2]:
search_place = 'Campinas'

In [3]:
dict_add_type = {
    0: 'Compra',
    1: 'Aluguel',
    2: 'Imóvel Novo',
}

In [4]:
# add_type = 0
# add_type = 1
add_type = 2

In [5]:
# driver = webdriver.Firefox(executable_path = GeckoDriverManager().install())
driver = webdriver.Firefox(service = Service(GeckoDriverManager().install()))

[WDM] - Downloading: 16.2kB [00:00, 7.22MB/s]                   


In [6]:
driver.get('https://www.vivareal.com.br/')

In [7]:
driver.find_element(By.CLASS_NAME, 'cookie-notifier__cta').click()

In [8]:
driver \
    .find_element(By.CLASS_NAME, 'js-select-business') \
    .find_elements(By.CLASS_NAME, 'item')[add_type].click()

In [9]:
driver.find_element(By.CLASS_NAME, 'js-select-type').find_elements(By.TAG_NAME, 'option')[0].click()

In [10]:
driver.find_element(By.ID, 'filter-location-search-input').send_keys(search_place)

In [13]:
time.sleep(2)

driver \
    .find_element(By.CLASS_NAME, 'autocomplete__list') \
    .find_elements(By.CLASS_NAME, 'autocomplete__item')[0].click()

In [14]:
def get_data(page_number):
    time.sleep(5)
    page_data = []
    has_suggestions = 0

    results = driver \
        .find_element(By.CLASS_NAME, 'results-list') \
        .find_elements(By.XPATH, './div')

    for item in results:
        if(item.get_attribute('data-type') == 'nearby'):
            has_suggestions = 1
            break
        elif item.get_attribute('data-type') != 'property':
            pass
        else:
            card = item.find_element(By.TAG_NAME, 'article')
            address = card.find_element(By.CLASS_NAME, 'property-card__address').text

            area = card \
                .find_element(By.CLASS_NAME, 'property-card__detail-area') \
                .find_element(By.CLASS_NAME, 'property-card__detail-value').text

            try:
                status = card.find_element(By.CLASS_NAME, 'property-card__construction-status').text
            except:
                status = '-'

            price = card \
                .find_element(By.CLASS_NAME, 'property-card__values') \
                .find_element(By.CLASS_NAME, 'property-card__price').text

            link = card \
               .find_element(By.CLASS_NAME, 'property-card__content-link').get_attribute('href')
            code = link.split('-')[-1][:-1]

            page_data.append([dict_add_type[add_type], address, area, price, status, code])

    return page_data, has_suggestions

In [15]:
results = []
page_number = 1
    
while True:
    print(f"Scraping page {page_number}...", end = '\n')
    page_data, has_sugestions = get_data(page_number)
    results.append(page_data)
    
    if(has_sugestions == 1):
        break
    else:
        btn_next_page = driver.find_elements(By.CLASS_NAME, 'js-change-page')[-1]

        if not btn_next_page.get_attribute('data-page'):
            break
        else:
            page_number += 1
            btn_next_page.click()

print('Done.')

Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Done.


In [16]:
results = [item for page_data in results for item in page_data]

In [17]:
driver.quit()

In [20]:
df = pd.DataFrame(
    data = results,
    columns = ['Tipo', 'Endereço', 'Área', 'Preço', 'Etiqueta', 'Código Viva Real']
)

In [21]:
df = df.replace('\n',' ', regex = True)

In [22]:
df.to_csv(f"outputs/{search_place} - {dict_add_type[add_type]}.csv", encoding = 'utf-8', index = False)